In [1]:
:e ImportQualifiedPost
:e FlexibleContexts
:e BlockArguments
:e TupleSections
:e FlexibleContexts
:e OverloadedStrings
:e LambdaCase

import Data.Map qualified as M
import qualified Data.Text as T
import Control.Arrow (first,second)
import Control.Monad
import Graphics.Vega.VegaLite hiding (density)
import qualified Graphics.Vega.VegaLite as VL
import IHaskell.Display.Hvega (vlShow)
import Control.Monad.State

:l Plotting.hs
:l ../src/Control/Monad/Bayes/Class.hs
:l ../src/Control/Monad/Bayes/Sampler.hs
:l ../src/Control/Monad/Bayes/Free.hs
:l ../src/Control/Monad/Bayes/Weighted.hs
:l ../src/Control/Monad/Bayes/Traced/Common.hs
:l ../src/Control/Monad/Bayes/Traced/Named.hs


We'll start with the example of a simple regression

In [2]:
paramPriorRegression = do
    slope <- traced "slope" $ normal 0 2
    intercept <- traced "intercept" $ normal 0 2
    noise <- traced "noise" $ gamma 4 4
    return (slope, intercept, noise)


-- regressionData :: (MonadSample m, Traversable t) => t Double -> m (t (Double, Double))
regressionData xs = do
    (slope, intercept, noise) <- paramPriorRegression
    forM xs \x -> do
        y <- normal (x*slope + intercept) (sqrt noise)
        return (x, y)

In [3]:

lower m = prior $ Control.Monad.Bayes.Weighted.hoist (flip evalStateT []) m


Line 2: Use section
Found:
(flip evalStateT [])
Why not:
(`evalStateT` [])

In [4]:

range = [-10,-9.9..10] :: [Double]
regressionSamples <- sampleIOfixed $ lower $ regressionData range
plotVega (fmap (second (T.pack . show)) (zip regressionSamples (Prelude.repeat "N/A")))


In [5]:
-- regression :: MonadInfer m => [Double] -> [Double] -> m (Double, Double, Double)
regression xs ys = do
    params@(slope, intercept, noise) <- paramPriorRegression
    forM_ (zip xs ys) \(x, y) -> factor $ normalPdf (slope * x + intercept) (sqrt noise) y
    return (slope, intercept, noise)


In [6]:
import Statistics.Distribution.Normal (normalDistr)
import qualified Statistics.Distribution as S (quantile, cumulative)
import Data.Maybe (fromMaybe)

prop c = do
  -- let oldSlope = fromMaybe undefined $ M.lookup ["slope"] c
  -- newSlope <- max 0.0000000001 . min 1 <$> normal oldSlope 0.01
  -- let oldIntercept = fromMaybe undefined $ M.lookup ["intercept"] c
  -- newIntercept <- max 0.0000000001 . min 1 <$> normal oldIntercept 0.01
  -- slope <- random
  -- intercept <- random
  -- return $ M.fromList [(["slope"], newSlope), (["intercept"], newIntercept)]
  key <- uniformD $ M.keys c
  -- traceM $ show ("foo", key)
  let old = fromMaybe undefined $ M.lookup key c
  new <- max 0.0000000001 . min 1 <$> normal old 0.05
  -- s <- random
  return $ M.insert key new c

-- (const $ return $ M.fromList [
--         (["slope"], S.cumulative (normalDistr 0 2) (-0.75)),
--         (["intercept"], S.cumulative (normalDistr 0 2) 0.0)
--         ]) 

mhRunsRegression <- sampleIOfixed $ prior $ mh 
    prop 200 $ regression range (snd <$> regressionSamples)
plotVega (range, (\(s,i,_) -> (s,i)) $ head mhRunsRegression) 

1.0
True
fromList [(["intercept"],0.7408640679453008),(["noise"],0.15936354678388287),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7408640679453008),(["noise"],0.15306046055429062),(["slope"],2.481036288296201e-2)]
0.92357428240712
True
fromList [(["intercept"],0.7408640679453008),(["noise"],0.15306046055429062),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7441993860731789),(["noise"],0.15306046055429062),(["slope"],2.481036288296201e-2)]
0.0
False
fromList [(["intercept"],0.7441993860731789),(["noise"],0.15306046055429062),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7441993860731789),(["noise"],0.15306046055429062),(["slope"],1.0e-10)]
2.771983694982357e-57
False
fromList [(["intercept"],0.7441993860731789),(["noise"],0.15306046055429062),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7441993860731789),(["noise"],0.15306046055429062),(["slope"],4.752526529357225e-2)]
1.0
True
fromList [(["intercept"],0.7441993860731789),([

This is a sample from the MCMC walk. Since this is an easy inference problem, it wasn't hard to generate good samples.

We can also view the posterior predictive, as follows:

In [7]:
-- posteriorPredictive :: MonadInfer m => [Double] -> [Double] -> m [Double]
posteriorPredictive xs ys = do
    (slope, intercept, noise) <- regression xs ys
    forM xs \x -> do
            let y' = x * slope + intercept
            normal y' (sqrt noise)


predictive <- head <$> (sampleIOfixed $ prior $ mh prop 100 $ posteriorPredictive range (snd <$> regressionSamples))
plotVega (fmap (second (T.pack . show)) (zip (zip range predictive) (Prelude.repeat "N/A")))


0.6752606751754328
False
fromList [(["intercept"],0.7408640679453008),(["noise"],0.15936354678388287),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7545821877834377),(["noise"],0.15936354678388287),(["slope"],2.481036288296201e-2)]
0.6329617817259844
True
fromList [(["intercept"],0.7408640679453008),(["noise"],0.15936354678388287),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7408640679453008),(["noise"],0.16962700244891724),(["slope"],2.481036288296201e-2)]
2.29073402417408e-9
False
fromList [(["intercept"],0.7408640679453008),(["noise"],0.16962700244891724),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7408640679453008),(["noise"],0.16962700244891724),(["slope"],3.303443077948489e-2)]
3.2037836771049576e-37
False
fromList [(["intercept"],0.7408640679453008),(["noise"],0.16962700244891724),(["slope"],2.481036288296201e-2)]
fromList [(["intercept"],0.7408640679453008),(["noise"],0.16962700244891724),(["slope"],4.273046515091616e-2)]
3.489293

# Traced proposal for linear regression with outliers

Now onto the harder problem, where a customized proposal is more useful.

In [8]:
paramPrior = do
    slope <- traced "slope" $ normal 0 2
    intercept <- traced "intercept" $ normal 0 2
    noise <- traced "noise" $ gamma 1 1
    prob_outlier <- traced "prob_outlier" $ uniform 0 0.5 
    return (slope, intercept, noise, prob_outlier)

forward (slope, intercept, noise, probOutlier) x = do
    isOutlier <- bernoulli probOutlier
    let meanParams = if isOutlier
                    then (0, 20)
                    else (x*slope + intercept, sqrt noise)
    return (meanParams, isOutlier)

-- regressionWithOutliersData :: (MonadSample m, Traversable t) => t Double -> m (t ((Double, Double), Bool))
regressionWithOutliersData xs = do
    params <- paramPrior

    forM (zip xs [0..]) \(x,i) -> do
        ((mu, std), isOutlier) <- traced (T.pack $ show i) $ forward params x
        y <- normal mu std
        return ((x, y), isOutlier)

In [39]:
range = [-10,-9.9..10] :: [Double]
samples <- sampleIOfixed $ lower $ regressionWithOutliersData range
plotVega (fmap (second (T.pack . show)) samples)

In [40]:
-- regressionWithOutliers :: (MonadSample m, MonadCond m) =>
--     [Double] -> [Double] -> m ((Double, Double, Double, Double), [Bool])
regressionWithOutliers xs ys = do
    params <- paramPrior
    
    outliers <- forM (zip3 xs ys [0..]) \(x, y, i) -> do
        ((mu, std), isOutlier) <- traced (T.pack $ show i) $ forward params x
        factor $ normalPdf mu std y
        return isOutlier
    return (params, outliers)

In [41]:


outlierProb s = (\(x, y) -> (\x -> log (x)) (fromIntegral y / (fromIntegral x))) 
        <$> foldr 
    (\(_,lb) li -> 
        [ if b then (num1+1, num2) else (num1,num2+1) | (b,(num1, num2)) <- zip lb li]) 
    (Prelude.repeat (0,0)) s


Line 3: Avoid lambda
Found:
(\ x -> log (x))
Why not:
logLine 3: Redundant bracket
Found:
(x)
Why not:
xLine 3: Redundant bracket
Found:
fromIntegral y / (fromIntegral x)
Why not:
fromIntegral y / fromIntegral x

In [42]:
import Data.Maybe (fromMaybe)
import Debug.Trace

simpleProposal oldProposal = do
  updateSlope <- bernoulli 0.1
  if updateSlope then 
    -- cleverProp oldProposal
    do 
      traceM "update slope"
      newSlope <- random
      newIntercept <- random
      newNoise <- random
      return $ M.union (M.fromList [
        (["slope"], newSlope), 
        (["intercept"], newIntercept), 
        (["noise"], newNoise)]) oldProposal
  else do 
    i <- uniformD [0..length (M.keys oldProposal)]
    -- traceM $ "update " <> show i
    val <- random
    -- vals <- replicateM (length (M.keys oldProposal) - 3) random
    return $ M.union (M.fromList $ zip [[T.pack $ show i]] [val]) oldProposal


cleverProp c = do
  -- let oldSlope = fromMaybe undefined $ M.lookup ["slope"] c
  -- newSlope <- max 0.0000000001 . min 1 <$> normal oldSlope 0.01
  -- let oldIntercept = fromMaybe undefined $ M.lookup ["intercept"] c
  -- newIntercept <- max 0.0000000001 . min 1 <$> normal oldIntercept 0.01
  -- slope <- random
  -- intercept <- random
  -- return $ M.fromList [(["slope"], newSlope), (["intercept"], newIntercept)]
  key <- uniformD $ M.keys c
  -- traceM $ show ("foo", key)
  let old = fromMaybe undefined $ M.lookup key c
  new <- random -- max 0.0000000001 . min 1 <$> normal old 0.05
  -- s <- random
  return $ M.insert key new c

In [44]:
mhRuns = sampleSTfixed $ prior $ mh cleverProp 10000 $ regressionWithOutliers range (snd . fst <$> samples)
-- mhRuns
plotVega  $ take 5000 (zip (fst <$> samples) (outlierProb mhRuns))

## Customized proposals



<!-- We'll use the version of `mh` from `Control.Monad.Bayes.Traced.Named` which allows the us name random variables and provide a custom proposal which can target those variables.

In particular, we'll want to perform a series of updates to the outlier guesses, followed by a series of updates to the slope, intercept and noise guesses. This should result in much more efficient inference overall. -->

In [40]:
-- import Numeric.Log

-- countOutliersWithWeight :: [((a, [Bool]), Log Double)] -> [(Double, Double)]
-- countOutliersWithWeight = foldr 
--     (\((_,lb),w) li -> 
--         [ if b then (num1+ 1, num2) else (num1,num2+ 1) | ((b),(num1, num2)) <- zip lb li]) 
--     (Prelude.repeat (0,0))

-- predData = baseData . 
--   dataColumn "Outlier Prediction" 
--       -- (Booleans $ (\((_,s),_) -> s) (maximumBy (compare `on` (snd)) smcRuns))
--       (Booleans ((\(x, y) -> ( if x > y then False else True) )
--         <$> (countOutliers (fst <$> smcRuns))))
  
-- predEncoding = baseEncoding . color [ MName "Outlier Prediction", VL.MmType VL.Quantitative]
-- showPlot predEncoding predData

Line 6: Redundant bracket
Found:
(b)
Why not:
bLine 12: Redundant if
Found:
if x > y then False else True
Why not:
not (x > y)Line 13: Redundant bracket
Found:
(\ (x, y) -> (if x > y then False else True))
  <$> (countOutliers (fst <$> smcRuns))
Why not:
(\ (x, y) -> (if x > y then False else True))
  <$> countOutliers (fst <$> smcRuns)

: 